### 1. Load all libraries we need to have in this section:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras


print(tf.__version__)


2.15.0


### 2. Load and Inspect Data

In [2]:
import pandas as pd

# Load your CSV data
df = pd.read_csv('cleaned_dataset.csv')

# Inspect the data
print(df.head())

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3744\4105609030.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_dataset.csv')


   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2             1    1             1  2020-03-05      2.0        1.0   
1      2             1    2             1  2020-03-06      2.0        1.0   
2      2             1    2             2  2020-09-06      1.0        2.0   
3      2             1    1             1  2020-12-06      2.0        2.0   
4      2             1    2             1  2020-06-21      2.0        2.0   

    AGE  PREGNANT  DIABETES  ...  INMSUPR  HIPERTENSION  OTHER_DISEASE  \
0  65.0       2.0       2.0  ...      2.0           1.0            2.0   
1  72.0       2.0       2.0  ...      2.0           1.0            2.0   
2  55.0       2.0       1.0  ...      2.0           2.0            2.0   
3  53.0       2.0       2.0  ...      2.0           2.0            2.0   
4  68.0       2.0       1.0  ...      2.0           1.0            2.0   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO  CLASIFFICATION_FINAL  ICU  \
0          

### 3 . Preprocess the Data


In [28]:
print(df.columns)

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU', 'DEAD'],
      dtype='object')


In [30]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Read the dataset
df = pd.read_csv('cleaned_dataset.csv')

# Extract features (X) and target variable (y)
X = df.drop(['AGE', 'DIABETES', 'DEAD'], axis=1)
y = df['DEAD']

# Identify numerical columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Extract relevant features from the date column
X['DATE_DIED'] = pd.to_datetime(X['DATE_DIED'])
X['year'] = X['DATE_DIED'].dt.year
X['month'] = X['DATE_DIED'].dt.month
X['day'] = X['DATE_DIED'].dt.day

# Drop the original date column
X = X.drop(['DATE_DIED'], axis=1)

# One-hot encode categorical variables
X = pd.get_dummies(X, columns=['SEX'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numerical columns after one-hot encoding
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Create a column transformer to standardize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols)
    ])



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3744\4267729199.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_dataset.csv')


### 4. Build the Neural Network Model

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Fit and transform the data
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### 5. Train the Model:

In [32]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
20972/20972 [==============================] - 74s 3ms/step - loss: 0.2817 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 2/10
20972/20972 [==============================] - 60s 3ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 3/10
20972/20972 [==============================] - 53s 3ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 4/10
20972/20972 [==============================] - 55s 3ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 5/10
20972/20972 [==============================] - 70s 3ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 6/10
20972/20972 [==============================] - 54s 3ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0.2626 - val_accuracy: 0.9265
Epoch 7/10
20972/20972 [==============================] - 48s 2ms/step - loss: 0.2628 - accuracy: 0.9264 - val_loss: 0

### 6. Evaluate the Model:

In [33]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


6554/6554 [==============================] - 12s 2ms/step - loss: 0.2605 - accuracy: 0.9273
Test Accuracy: 92.73%


### 7. Fine-Tune and Experiment

Based on the initial results, we can fine-tune our model. Experiment with different architectures, activation functions, and hyperparameters. Consider using techniques like dropout or regularization to avoid overfitting.

### 8. Save and Use the Model

In [34]:
model.save('Neural_Model.h5')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
